### Products details

- Products listed in a ecomerce website have different filters that enable a user ro find what they are looking for.
 An example is when a user visits Jumia and he/she is looking for  a watch to buy, the first thing they would do is 
 to filter the products listed on the page by the product name. This narrows down the products but leaves a wide
 variety of products to choose from varying in sizes , prices, brands, origin etc. To accomodate such variations in 
 products , the developers needed to come up with a system to handle products and their variations. The above called for 
 a system that is able to handle products fillters dynamically.


 - LETS TRY TO SOLVE THIS TRADITIONALLY.
    The intuitive solution would to make table that store products attributes and have many to many relationships linking the products and attrubutes.
    eg I have listed `LM392` for 2$. One can easily find it on the platform if they have that specifid part number cause they would only need to search for it.
    some common attributes it would have are color , size ,thermal resistivity, voltage offsets and many more.  If i was to make a table for each attribute the product 
    have i would atleast need 5 tables. Ten days latter , i want to enlist a new product and this time, it a wearable item like Airforce . The products would come with its
    own attriburtes like color, size, age, best_for, etc. this would increase the columns by not less than 4. This indicates that the number of tables would be increasing 
    as i add more products and the querying would become more complicated as time progeresses.

- PROPOSED SOLUTION
  1. Each product attribute is recorded in a table of attributes. 
     Each entry has  a relationship to a table which records the products attrubutesin details.
     Example:
        product A is a certain shoe from brand x. Its size is 42-39. Black in color and is mede of leather.
            ITEMS TO BE RECORDED IN THE ATTRIBUTE TABLE:
                1. Brands
                    1.1 Brand x

                2. Size
                    30-33
                    39-42 
                3. Color
                    Black
                    Red
                    ...
                4. Material
                    Leather
        Product B is my LM392: Its made of silcon,its size is 67 by 45 mm, its black 
            I do not need to creaate any new attribute entry since all of them are in the db. I only need to record the specifics.
                eg  In the attributes specifics is would records silcon and link it with materials.\
                    67 by 45 and link it to size.
                    ......

- HOW WOULD QUERYING BE CONDUCTED
    - For  a start a user visits our site and want to buy the LM392. He/she searches for that specific part number but is presented with 
        a variery of choices which would take time to fiter manually. On the side bar they have check boxes for size ,manufacurer, price,
        color etc. If they check on of the boxes and click apply filter, a reqeust should be sent to filter the products on
            1. The attrubutes selected 
            2. The products whose specific attributes meet the one selected.
                eg :
                    size 12 by 13
                    price range 
            
            - THis can be achived by looping through the procuts and filtering each specific attribute.
Ean example is shown below
    ORM relainships



          | `products`| `GN-ATTR`| `SP-ATTR`|
__________|___________|__________|__________|
_________ |___________| _________|__________|
`Products`|NONE       |  `MANY->`| `many->` |
          |           |  `MANY`  | `many`   |
__________|___________|__________|__________|
`GN-ATTR` |           |          | `one->`  |
          | `many->`  | NONE     | `many`   |
          | `many`    |          |          |
__________|___________|__________|__________|
`SP-ATTR` | `many->`  | `many->` |  NONE    |
          |  `many`   | `one`    |          |
__________|___________|__________|__________|

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from datetime import datetime

Base = declarative_base()
engine = create_engine("sqlite:///ecommerce.db")

# ---- Association Tables ----
product_attribute_values = Table(
    'product_attribute_values',
    Base.metadata,
    Column('product_id', Integer, ForeignKey('products.id'), primary_key=True),
    Column('attribute_value_id', Integer, ForeignKey('attribute_values.id'), primary_key=True)
)

product_attribute_link = Table(
    'product_attribute_links',
    Base.metadata,
    Column('product_id', Integer, ForeignKey('products.id'), primary_key=True),
    Column('attribute_id', Integer, ForeignKey('attributes.id'), primary_key=True)
)

attribute_value_link = Table(
    'attribute_value_links',
    Base.metadata,
    Column('attribute_id', Integer, ForeignKey('attributes.id'), primary_key=True),
    Column('value_id', Integer, ForeignKey('attribute_values.id'), primary_key=True)
)

# ---- Models ----
class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    price = Column(Integer)  # Price in cents
    
    # Relationships
    attributes = relationship(
        "Attribute", 
        secondary=product_attribute_link,
        back_populates="products"
    )
    attributes_values = relationship(
        'AttributeValue',
        secondary=product_attribute_values,
        back_populates='products'
    )
    def to_dict(self):
        return {
            "name":self.name,
            "price":self.price
        }
    
    def __repr__(self):
        return f"<Product(id={self.id}, name='{self.name}', price={self.price})>"

class Attribute(Base):
    __tablename__ = 'attributes'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)  # "color", "size", etc.
    data_type = Column(String)  # "string", "number", "size_range"
    
    # Relationships
    products = relationship(
        "Product", 
        secondary=product_attribute_link,
        back_populates="attributes",
        lazy='dynamic'
    )
    values = relationship(
        "AttributeValue",
        secondary=attribute_value_link,
        back_populates="attributes"
    )
    def __repr__(self):
        return f"<Attribute(id={self.id}, name='{self.name}', data_type='{self.data_type}')>"



class AttributeValue(Base):
    __tablename__ = 'attribute_values'
    id = Column(Integer, primary_key=True)
    value = Column(String, nullable=False)  # "red", "39-42", "XL"
    unit = Column(String)  # "mm", "inches", "EU" (optional)
    
    # Relationships
    attributes = relationship(
        "Attribute", 
        secondary=attribute_value_link,
        back_populates="values"
    )
    products = relationship(
        'Product',
        secondary=product_attribute_values,
        back_populates='attributes_values',lazy='dynamic'
    )
    def __repr__(self):
        if self.unit:
            return f"<AttributeValue(id={self.id}, value='{self.value}', unit='{self.unit}')>"
        return f"<AttributeValue(id={self.id}, value='{self.value}')>"

# Create tables
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)


In [ ]:
from random import randint
import random
def seed_products():
    session = Session()

    try:
        # ===== CREATE CORE ATTRIBUTE VALUES =====
        # Brand values
        samsung_val = AttributeValue(value="Samsung")
        apple_val = AttributeValue(value="Apple")
        nike_val = AttributeValue(value="Nike")
        adidas_val = AttributeValue(value="Adidas")

        # Color values
        black_val = AttributeValue(value="black")
        blue_val = AttributeValue(value="blue")
        red_val = AttributeValue(value="red")
        white_val = AttributeValue(value="white")
        silver_val = AttributeValue(value="silver")

        # Size values
        size_39_42 = AttributeValue(value="39-42", unit="EU")
        size_xl = AttributeValue(value="XL", unit="US")
        size_5x5 = AttributeValue(value="5x5", unit="mm")
        size_15_6 = AttributeValue(value="15.6", unit="inch")

        # Material values
        leather_val = AttributeValue(value="Leather")
        polyester_val = AttributeValue(value="Polyester")
        silicon_val = AttributeValue(value="Silicon")
        cotton_val = AttributeValue(value="Cotton")

        # Add all values to session
        session.add_all([
            samsung_val, apple_val, nike_val, adidas_val,
            black_val, blue_val, red_val, white_val, silver_val,
            size_39_42, size_xl, size_5x5, size_15_6,
            leather_val, polyester_val, silicon_val, cotton_val
        ])
        session.flush()

        # ===== CREATE ATTRIBUTES AND LINK VALUES =====
        brand_attr = Attribute(name="brand", data_type="string")
        color_attr = Attribute(name="color", data_type="string")
        size_attr = Attribute(name="size", data_type="size_range")
        material_attr = Attribute(name="material", data_type="string")

        # Link values to attributes
        brand_attr.values = [samsung_val, apple_val, nike_val, adidas_val]
        color_attr.values = [black_val, blue_val, red_val, white_val, silver_val]
        size_attr.values = [size_39_42, size_xl, size_5x5, size_15_6]
        material_attr.values = [leather_val, polyester_val, silicon_val, cotton_val]

        # Add attributes to session
        session.add_all([brand_attr, color_attr, size_attr, material_attr])
        session.flush()

        # ===== CREATE PRODUCTS WITH ATTRIBUTES AND VALUES =====
        # Samsung products
        galaxy_s23 = Product(
            name="Galaxy S23",
            price=99900,
            attributes=[brand_attr, color_attr, size_attr],
            attributes_values=[samsung_val, black_val, size_5x5]
        )
        
        galaxy_watch = Product(
            name="Galaxy Watch",
            price=29900,
            attributes=[brand_attr, color_attr],
            attributes_values=[samsung_val, silver_val]
        )

        # Apple products
        iphone = Product(
            name="iPhone 15",
            price=109900,
            attributes=[brand_attr, color_attr],
            attributes_values=[apple_val, blue_val]
        )

        # Nike products
        running_shoes = Product(
            name="Running Shoes",
            price=15900,
            attributes=[brand_attr, size_attr, material_attr],
            attributes_values=[nike_val, size_39_42, polyester_val]
        )

        # Adidas products
        football_cleats = Product(
            name="Football Cleats",
            price=13900,
            attributes=[brand_attr, size_attr, material_attr],
            attributes_values=[adidas_val, size_39_42, leather_val]
        )

        # Add more interconnected products
        products = [
            galaxy_s23, galaxy_watch, iphone, 
            running_shoes, football_cleats
        ]

        # Add 10 more Samsung products with random attributes
        for i in range(1, 11):
            product = Product(
                name=f"Samsung Product {i}",
                price=random.randint(5000, 50000),
                attributes=[brand_attr, random.choice([color_attr, size_attr, material_attr])],
                attributes_values=[samsung_val, random.choice([black_val, blue_val, size_5x5, size_15_6, polyester_val])]
            )
            products.append(product)

        session.add_all(products)
        session.commit()

        print(f"✅ Successfully seeded {len(products)} products")
        print(f"   - Samsung products: {len([p for p in products if samsung_val in p.attributes_values])}")
        print(f"   - Apple products: {len([p for p in products if apple_val in p.attributes_values])}")
        print(f"   - Nike products: {len([p for p in products if nike_val in p.attributes_values])}")
        print(f"   - Adidas products: {len([p for p in products if adidas_val in p.attributes_values])}")

    except Exception as e:
        session.rollback()
        print(f"❌ Seeding failed: {e}")
        raise
    finally:
        session.close()

if __name__ == "__main__":
    seed_products()

In [ ]:
from contextlib import contextmanager
from time import perf_counter
@contextmanager
def timeme():
    star = perf_counter()
    print("Strartin query: ")
    ses = Session()
    try:
        yield ses 
    except Exception as e:
        print(e)
    finally:
        print("Time taken = ",perf_counter()-star ,'seconds')
        print("Done. ")
        ses.close()

In [ ]:
with timeme() as db_session:
    prod = db_session.query(Product).filter_by(id=4).first()
    print(prod.attributes_values)

In [ ]:
## Get a specific attribute ,and query a certain limit of products
## usefull for pagination
with timeme() as db_session:
    attributes = db_session.query(Attribute).filter(
        Attribute.name=='brand').first()
    print(attributes)
    print(attributes.products.limit(4).all())

In [ ]:
## construct the different brands and their asscociated products
## Limit the products to 3
PRODUCTS_LIMIT = 3
from pprint import pprint
with timeme() as db_session:
    brands = db_session.query(Attribute).filter(
        Attribute.name=='brand'
    ).first().values
    products_details = {brand.value:[i.to_dict() for i in brand.products.limit(PRODUCTS_LIMIT).all()] for brand in brands}
    pprint(products_details)

In [ ]:
## Filter by some specific attribute value
## eg shoe size XL
test_values = ['XL','silicon' ,'red',"39-42"]
def fetch_multiple_values(val):
    print(f"Val={val}")
    with timeme() as db_session:
        attribute = db_session.query(AttributeValue).filter_by(value=val).first()
        product = None if not attribute else attribute.products.limit(PRODUCTS_LIMIT).all()
        print(product)
[fetch_multiple_values(i) for i in test_values]

In [ ]:
## Filter by a certain attribute
## Eg filter products which have the attribute material
attribute_name = 'material'
with timeme() as db_session:
    attr = db_session.query(Attribute).filter_by(name=attribute_name).first()
    products = [i.to_dict() for i in attr.products.limit(PRODUCTS_LIMIT).all()]
    pprint(products)

In [ ]:
## List features one can filter with
## Provide the data to construct the filters with
with timeme() as db_session:
    attributes = db_session.query(Attribute).all()
    attributes_values = {attr.name :[i.value for i in attr.values] for attr in attributes}
    pprint(attributes_values)

In [ ]:
product_filters ={'brand': ['Samsung'],
                'color': ['black', 'blue'],
                'price_range': (10000, 50000)}

with timeme() as db_session:
    ps = []
    query =db_session.query(Product)
    for filter_q ,value in product_filters.items():
        if filter_q == 'price_range':
            start ,end = value.split("-")
            prod = query.filter(
                Product.price >= int(start) ,Product.price <=int(end)
            ).first()
            if not prod:
                break

        
        prod = query.filter(
            Product.attributes
        )
       

ps 

In [ ]:
from sqlalchemy.orm import joinedload
from contextlib import contextmanager
from time import perf_counter
from pprint import pprint

class ProductQuery:
    def __init__(self, session_factory):
        self.Session = session_factory
    
    @contextmanager
    def _get_session(self):
        session = self.Session()
        try:
            yield session
        except Exception as e:
            session.rollback()
            raise e
        finally:
            session.close()
    
    @contextmanager
    def timed_query(self):
        start = perf_counter()
        print("Starting query...")
        with self._get_session() as session:
            try:
                yield session
            finally:
                print(f"Time taken: {perf_counter() - start:.4f} seconds")
    
    # Core query methods
    def get_product(self, product_id):
        """Get a product by ID with all attributes"""
        with self._get_session() as session:
            return session.query(Product).filter_by(id=product_id).first()
    
    def get_products_by_attribute_value(self, attr_name, value, limit=None):
        """Get products filtered by attribute value"""
        with self._get_session() as session:
            query = session.query(Product).join(
                Product.attributes_values
            ).join(
                AttributeValue.attributes
            ).filter(
                Attribute.name == attr_name,
                AttributeValue.value == value
            )
            if limit:
                query = query.limit(limit)
            return query.all()
    
    def get_products_by_brand(self, brand_name, limit=None):
        """Specialized brand filter"""
        return self.get_products_by_attribute_value('brand', brand_name, limit)
    
    def get_filter_options(self):
        """Get all available filter attributes and values"""
        with self._get_session() as session:
            attributes = session.query(Attribute).options(
                joinedload(Attribute.values)
            ).all()
            return {
                attr.name: [val.value for val in attr.values]
                for attr in attributes
            }
    
    def get_products_with_attributes(self, attribute_names, limit=None):
        """Get products that have specific attributes"""
        with self._get_session() as session:
            query = session.query(Product).join(
                Product.attributes
            ).filter(
                Attribute.name.in_(attribute_names)
            )
            if limit:
                query = query.limit(limit)
            return query.all()
    
    def get_brand_products_summary(self, limit_per_brand=3):
        """Get products grouped by brand with pagination"""
        with self._get_session() as session:
            brand_attr = session.query(Attribute).filter_by(name='brand').first()
            return {
                brand.value: [p.to_dict() for p in brand.products.limit(limit_per_brand)]
                for brand in brand_attr.values
            }

    # Advanced filtering
    def filter_products(self, filters, limit=None):
        """
        Complex filtering with multiple criteria
        filters = {
            'brand': ['Samsung'],
            'color': ['black', 'blue'],
            'price_range': (10000, 50000)
        }
        """
        with self._get_session() as session:
            query = session.query(Product)
            
            # Handle attribute filters
            for attr_name, values in filters.items():
                if attr_name == 'price_range':
                    min_price, max_price = values
                    query = query.filter(Product.price.between(min_price, max_price))
                else:
                    query = query.join(
                        Product.attributes_values
                    ).join(
                        AttributeValue.attributes
                    ).filter(
                        Attribute.name == attr_name,
                        AttributeValue.value.in_(values)
                    )
            
            if limit:
                query = query.limit(limit)
            
            return query.all()

# Usage Example
if __name__ == "__main__":
    query = ProductQuery(Session)
    
    # Example queries
    print("=== Samsung Products ===")
    samsung_products = query.get_products_by_brand("Samsung", limit=3)
    pprint([p.to_dict() for p in samsung_products])
    
    print("\n=== Filter Options ===")
    filters = query.get_filter_options()
    pprint(filters)
    
    print("\n=== Products with size 39-42 ===")
    size_products = query.get_products_by_attribute_value("size", "39-42")
    pprint([p.to_dict() for p in size_products])
    
    print("\n=== Complex Filter ===")
    filtered = query.filter_products({
        'brand': ['Samsung', 'Apple'],
        'price_range': (10000, 100000)
    }, limit=5)
    pprint([p.to_dict() for p in filtered])

In [6]:
from pathlib import Path
import sys 
from pprint import pprint 
app_folder = Path().cwd().parent

sys.path.append(str(app_folder.absolute()))

from app.data_manager.product.attribute_manager import ProductAttributeManager
from app.data_manager.product.product_manager import ProductQuery

In [3]:
# Create a new product
product_data = {
    "name": "New Productl",
    'vendor_id':2,
    "description": "This is a new product89",
    "price": 458,
    "stock": 100,
    "category": "Electronics",
    "image_url": "http://example.com/image.jpg",
    "preview_url": "http://example.com/preview.jpg",
}

product_id = ProductAttributeManager.create_charge_product(product_data)
print(product_id)
# Update product attributes
ProductAttributeManager.update_product_attributes(product_id, {
    'color': ['red', 'blue'],
    'size': ['small', 'medium', 'large']
})

# Get product attributes
attributes = ProductAttributeManager.get_product_attributes(product_id)

{'type': <ChargeType.PRODUCT: 'Product'>, 'recipient_id': 21}
21


In [7]:
g = ProductQuery().get_product(
    product_id=21
)
pprint(g)

{'attribute_values': [{'attribute_types': [], 'unit': None, 'value': 'red'},
                      {'attribute_types': [], 'unit': None, 'value': 'blue'},
                      {'attribute_types': [], 'unit': None, 'value': 'small'},
                      {'attribute_types': [], 'unit': None, 'value': 'medium'},
                      {'attribute_types': [], 'unit': None, 'value': 'large'}],
 'attributes': {'color': [], 'size': []},
 'category': 'Electronics',
 'description': 'This is a new product',
 'discount': 15,
 'hiden_id': 'KR',
 'id': 21,
 'image_src': 'http://example.com/image.jpg',
 'in_stock': 100,
 'name': 'New Productl',
 'original_price': Decimal('578.00'),
 'price': Decimal('491.31')}


{'id': 21, 'name': 'New Productl', 'price': Decimal('491.31'), 'category': 'Electronics', 'image_src': 'http://example.com/image.jpg', 'hiden_id': 'KR', 'discount': 15, 'original_price': Decimal('578.00'), 'description': 'This is a new product', 'in_stock': 100, 'attributes': {'color': [], 'size': []}, 'attribute_values': [{'value': 'red', 'unit': None, 'attribute_types': []}, {'value': 'blue', 'unit': None, 'attribute_types': []}, {'value': 'small', 'unit': None, 'attribute_types': []}, {'value': 'medium', 'unit': None, 'attribute_types': []}, {'value': 'large', 'unit': None, 'attribute_types': []}]}
